> **Atividade** (Sendo aprimorada):
> 1. Adapte o código acima para fazer com que a função `solve_triangular_superior()` também retorno o número de operações (somas, produtos, divisões e subtrações numéricas) que foram realizadas. (**Dica**: A resposta é $n^2$; tente deduzir este valor formalmente).

In [533]:
import numpy as np

In [534]:
def solve_triangular_superior(U, b):
    '''Resolve um sistema triangular superior do tipo Ux = b.

    Parametros obrigatorios
    ----------
    U : Array-like de dimensao 2
        Matriz quadrada triangular superior inversível

    b : Array-like de dimensão 1
        Vetor independente

    Saída
    ----------
    x : Array-like de dimensão 1
        Solução do sistema Ux = b
    
    num_op : int 
        Número de operações'''

    num_op = 0
    n = U.shape[0]          # Ordem das matrizes
    
    # Cópias usuais para evitar problemas
    x = b.copy().reshape(n)

    # Vai linha-a-linha, de baixo para cima, escalonando a matriz utilizando o pivô na diagonal
                        
    for i in range(n-1,-1,-1):
        x[i] /= U[i,i]     # Normaliza a i-ésima linha

        num_op +=1
        for j in range(i-1,-1,-1):
            x[j] -= U[j,i]*x[i]     # Pivoteia a i-ésima coluna, utilizando a entrada diagonal como pivô
            num_op += 2

    return x, num_op

In [535]:
tabela = []
ordem = 5
for tam in range(2, ordem+1):
    U = 20*np.random.rand(tam,tam) -10 # Matriz triangular aleatória com entradas em [-10,10]

    for i in range(tam):
        for j in range(i):
            U[i,j]=0.0        # Aniquila as entradas abaixo da diagonal principal para deixar triangular superior    
            
    b = 20*np.random.rand(tam) - 10 # Vetor aleatório com entradas em [-10,10]

    x, num_operacoes = solve_triangular_superior(U,b) # Solução pretendida do sistema Ux = b
    tabela.append([tam, num_operacoes])
    # print(f"\nA matriz U é dada por\n{U}")
    # print(f"\nO vetor b é dada por\n{b}")
    # print(f"\nO vetor x é dada por\n{x}")
    # print(f"\nO vetor Ux é dada por\n{U@x}")
    # print(f"---------------------------------")

print(f"\nTabela relacionando a Ordem da matriz e o número de operações")
print(f"\n|{'Ordem':^10}|{'Num. Op.':^10}|")

for linha in tabela:    
    print(f"|{linha[0]:^10}|{linha[1]:^10}|")


Tabela relacionando a Ordem da matriz e o número de operações

|  Ordem   | Num. Op. |
|    2     |    4     |
|    3     |    9     |
|    4     |    16    |
|    5     |    25    |


> 2. Adapte o código acima para fazer o processo análogo (também contando operações), mas com matrizes triangulares _inferiores_.

In [536]:
def solve_triangular_inferior(L, b):
    '''Resolve um sistema triangular inferior do tipo Lx = b.

    Parametros obrigatorios
    ----------
    L : Array-like de dimensao 2
        Matriz quadrada triangular inferior inversível

    b : Array-like de dimensão 1
        Vetor independente

    Saída
    ----------
    x : Array-like de dimensão 1
        Solução do sistema Lx = b
    
    num_op : int 
        Número de operações'''

    num_op = 0
    n = L.shape[0]          # Ordem das matrizes
    
    # Cópias usuais para evitar problemas
    x = b.copy().reshape(n)
                        
    for i in range(n):
        for j in range(i):
            x[i] -= L[i][j] * x[j]
            num_op += 2
        x[i] /= L[i][i]
        num_op += 1
    return x, num_op

In [537]:
ordem = 3
L = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
# print (U)
for i in range(ordem):
    for j in range(ordem-1, i, -1):
        L[i,j]=0.0        # Aniquila as entradas abaixo da diagonal principal para deixar triangular superior
        
b = 20*np.random.rand(ordem) - 10 # Vetor aleatório com entradas em [-10,10]

x, num_operacoes = solve_triangular_inferior(L,b) # Solução pretendida do sistema Ux = b

print("Nº Op.: ", num_operacoes)
print(f"\nO vetor L@x é dada por\n{L@x}")
print(f"\nO vetor b é dada por\n{b}")

Nº Op.:  9

O vetor L@x é dada por
[ 1.09169198  6.89847543 -8.24560988]

O vetor b é dada por
[ 1.09169198  6.89847543 -8.24560988]


> 3. Adapte o algoritmo de Crout&ndash;Dolittle, para fazer com que ele também retorne o número de operações realizadas.

In [538]:
def crout_dolittle(A):
    ''' Decomposição LU de A pelo algoritmo de Crout'''

    num_op = 0

    m,n = np.shape(A)

    L = np.zeros((n,n))
    U = np.zeros((n,n))

    L[0,0] = 1          # Escolha de Dolittle
    U[0,0] = A[0,0]

    for j in range(1,n):
        U[0,j] = A[0,j]/L[0,0]      # Determina a primeira linha de U
        L[j,0] = A[j,0]/U[0,0]      # Determina a primeira coluna de L
        num_op += 2
    
    for i in range(1,n):
        L[i,i]=1        # Escolha de Dolittle
        
        U[i,i] = A[i,i] - sum([L[i,k]*U[k,i] for k in range(i)])

        num_op += 1     # Referente ao '-'
        num_op += i-1   # Referente ao 'sum'
        num_op += i     # Referente ao '*' 

        for j in range(i+1,n):
            U[i,j] = (A[i,j] - sum([L[i,k]*U[k,j] for k in range(i)]))/L[i,i]       # Determina a i-ésima linha de U
            L[j,i] = (A[j][i] - sum([L[j,k]*U[k,i] for k in range(i)]))/U[i,i]      # Determina a i-ésima coluna de L
            num_op += 2*(i + (i-1) + 2)
    return L , U, num_op

In [539]:
tabela = []
ordem = 10
for tam in range(2, ordem+1):
    A = 20*np.random.rand(tam, tam) - 10.0       # Matriz aleatória com entradas entre -10 e 10

    L , U, num_operacoes = crout_dolittle(A)               # Decomposição LU de $A$

    tabela.append([tam, num_operacoes, int(2/3 * tam**3 - 2/3 * tam +1)])

    # print(f"A matriz A é dada por\n{A}")
    # print(f"\nA matriz L é dada por\n{L}")
    # print(f"\nA matriz U é dada por\n{U}")
    # print(f"\n\nO produto LU é dado por\n{L@U}")
    # print(f"---------------------------------")

print(f"\nTabela relacionando a Ordem da matriz e o número de operações")
print(f"\n|{'Ordem':^10}|{'Num. Op.':^10}|{'Equacao':^10}|")

for linha in tabela:    
    print(f"|{linha[0]:^10}|{linha[1]:^10}|{linha[2]:^10}|")


Tabela relacionando a Ordem da matriz e o número de operações

|  Ordem   | Num. Op. | Equacao  |
|    2     |    4     |    5     |
|    3     |    16    |    17    |
|    4     |    40    |    41    |
|    5     |    80    |    81    |
|    6     |   140    |   141    |
|    7     |   224    |   225    |
|    8     |   336    |   337    |
|    9     |   480    |   481    |
|    10    |   660    |   661    |


> 4. Utilize as funções criadas no passo anterior para criar uma função que recebe uma matriz $A$, um inteiro positivo $n$ e um vetor $b$ e
>     - Decompõe a matriz $A$ como $A=LU$.
>     - Resolve o sistema linear $A^n x = b$, com aplicações sucessivas das funções criadas nos passos 1. e 2.
>     - Conta o número de operações.

In [540]:
def decompoe_matriz (A, n, b):
    '''Função decompor matriz em produto LU e calcular o sistema A^n x = b
    
    Parametros obrigatorios
    ----------
    A : Array-like
        Matriz quadrada

    n : int
        Valor do expoente no cálculo

    b : Array-like de dimensão 1
        Vetor independente'''
    num_op_total = 0
    
    # Primeiro Item
    L, U, num_op = crout_dolittle(A)
    num_op_total += num_op

    # Segundo Item
    x = b.copy()
    for i in range(0, n):
        y, num_op = solve_triangular_inferior(L, x)
        num_op_total += num_op

        x, num_op = solve_triangular_superior(U, y)
        num_op_total += num_op
    return x, num_op_total

In [541]:
ordem = 3
A = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
b = 20*np.random.rand(ordem) - 10 # Vetor aleatório com entradas em [-10,10]
n = 20

x, num_operacoes = decompoe_matriz(A, n, b)
A_ = A.copy()

for i in range(1, n):
    A_ = A_@A

print(f"(A^{n})x = b")
print("Nº Op.: ", num_operacoes)
print(f"\n(A_)x : \n", A_@x)
print(f"\nb: \n",b)

(A^20)x = b
Nº Op.:  376

(A_)x : 
 [-1.49580479  5.27471161 -9.88705158]

b: 
 [-1.49580324  5.27470959 -9.88704956]


> 5. Adapte o algoritmo de resolução de sistemas lineares da aula 9 para que ele conte o número de operações realizadas.

In [542]:
def pivot_partial(x):
    return np.argmax(abs(x))

def ref(A, array_b, tol=None, pivot=pivot_partial, verbose=False):

    #Faz cópias
    T = np.array(A).astype(float)
    b = array_b.copy()              # Vetor de termos independentes

    posicao_pivos = []

    #Grava o tamanho
    n_linhas, n_colunas = np.shape(T)


    if tol == None:
        # Mesma que no Octave
        tol = 2**(-52) * np.max(abs(T)) * max(n_linhas, n_colunas)

    #Vê quantos pivôs já foram achados
    n_pivos = 0

    #Linhas na qual trabalharemos
    j = 0

    if verbose:
        print("Vamos escalonar parcialmente a matriz")
        print(T)

    num_op = 0
    while (j < n_colunas and n_pivos < n_linhas):
        if verbose:
            print("=====")
            print(f"Vamos pivotear a coluna {j}.")

        #Encontra o pivô
        p = pivot(T[n_pivos:, j]) + n_pivos
        num_op +=1

        if abs(T[p, j]) > tol:
            #Encontramos um pivô.
            #Troca linhas caso necessário
            if p != n_pivos:

                for k in range(j, n_colunas):
                    temp = T[p, k]
                    T[p, k] = T[n_pivos, k]
                    T[n_pivos, k] = temp

                if verbose:
                    print(
                        f"Precisamos trocar a linha {n_pivos} com a linha {p}."
                    )
                    print(T)

                p = n_pivos

            #Pivoteia abaixo
            for k in range(p + 1, n_linhas):
                if abs(T[k, j]) > tol:
                    multiplicador = T[k, j] / T[p, j]
                    num_op += 1
                    T[k, j + 1:] = T[k, j + 1:] - multiplicador * T[p, j + 1:]
                    T[k, j] = 0
                    num_op += n_colunas - 1 - j


                    b[k] = b[k] - multiplicador*b[j]
                    num_op +=2
                    
                    # print("T:",T)
                    # print("b:",b)
                    # print(f"({k}, {j})")

            if verbose:
                print("Aniquila as entradas abaixo:")
                print(T)


            #Conta o pivô a mais
            n_pivos += 1
            posicao_pivos.append(j)
        else:
            if verbose:
                print(f"A coluna {j} nao tem pivo.")

        j += 1

    if verbose:
        print(f"Numero de operacoes: {num_op}")

    return [T, b, posicao_pivos], num_op

def retrossub(A, array_b, pospiv=[], tol=None, verbose=False):
    
    num_op = 0

    b = array_b.copy()

    #Faz as cópias usuais para evitar problemas
    R = np.array(A).astype(float)
    m, n = np.shape(R)

    if tol == None:
        tol = 2**(-30) * np.max(abs(R))

    if verbose:
        print("Vamos fazer retrosubstituição na matriz")
        print(A)

    #caso nao saibamos as posições dos pivos, temos que encontrá-los
    if pospiv == []:
        if verbose:
            print('Nâo foi dada a posição dos pivôs da matriz.')
            print('Vamos encontrá-los:')

        i = 0
        j = 0
        while (i < m):
            while abs(R[i, j]) < tol and j < n:
                j += 1

            if j == n:
                break
            else:
                pospiv.append(j)
                i += 1

    if verbose:
        print(f"Os pivôs estao nas colunas {pospiv}")
        print()

    numero_de_pivos = len(pospiv)
    for i in range(numero_de_pivos - 1, -1, -1):
        if verbose:
            print('=====')

        #Normaliza o pivô, caso necessário
        if abs(R[i, pospiv[i]] - 1) > tol:
            # R[i, pospiv[i] + 1:] /= R[i, pospiv[i]]
            b[i] /= R[i, pospiv[i]]
            R[i, pospiv[i]] = 1
            if verbose:
                print(f"Vamos normalizar o pivo na linha {i}.")
                print(R)
                print()

        #Aniquila as entradas acima
        if verbose:
            print(f"Vamos aniquilar as entradas acima do pivo na posição ({i},{pospiv[i]}).")

        for k in range(i - 1, -1, -1):
            b[k] = b[k] - R[k, pospiv[i]]
            # R[k, pospiv[i] + 1:] -= R[k, pospiv[i]] * R[i, pospiv[i] + 1:]
            R[k, pospiv[i]] = 0

        if verbose:
            print("_"*10)
            print(R)

    return [R, b, pospiv], num_op


def rref(A, array_b, tol=None, pivot=pivot_partial, verbose=False):
    num_op_total = 0

    T, num_op = ref(A, array_b, tol=tol, pivot=pivot, verbose=verbose)
    num_op_total += num_op

    # print(f"\nA_superior: \n{T[0]}")
    # print(f"\nb: \n{T[1]}")


    T, num_op = retrossub(T[0], array_b, pospiv=T[2], tol=tol, verbose=verbose)
    num_op_total += num_op

    # print(f"\nA_identidade: \n{T[0]}")
    # print(f"\nb: \n{T[1]}")

    return T[1], num_op_total  # Retorna matriz b (igual a x) e nº de operações

In [543]:
ordem = 4
A = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
b = 20*np.random.rand(ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]

x, num_operacoes = rref(A, b, verbose=False)

print("Número de Operações: ", num_operacoes)

# print("A@x: ", A@x)
# print("b: ", b)

Número de Operações:  36


> 6. Adapte o algoritmo abaixo, que realiza o produto de duas matrizes $A$ e $B$, para que ele também conte o número de operações realizadas.

In [544]:
def matmul(A,B):
    """Produto de matrizes quadradas de mesma ordem

    Parametros obrigatorios
    ----------
    A , B : Array-like de dimensao 2
        Matrizes quadradas de mesma ordem

    Saída
    ----------
    C : Array-like de dimensão 2
        Produto C = AB
        
    Num_op : Numero de operacoes"""

    num_op = 0    
    n=A.shape[0]
    
    C = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i,j] += A[i,k]*B[k,j]
                num_op += 2
                
    return C, num_op

In [545]:
ordem = 3
A = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
B = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]

C, num_operacoes =  matmul(A, B)
print("Nº Op.: ", num_operacoes)
print(f"A@B: \n{A@B}")
print(f"\nC: \n{C}")

Nº Op.:  54
A@B: 
[[ 51.91796026 -33.64643438 -24.89435808]
 [-28.19596874 -19.4790324  -30.44257037]
 [ -5.62104653  40.41848566  -3.61668897]]

C: 
[[ 51.91796026 -33.64643438 -24.89435808]
 [-28.19596874 -19.4790324  -30.44257037]
 [ -5.62104653  40.41848566  -3.61668897]]


> 7. Crie uma matriz aleatória $A$ de ordem $4\times 4$ e um vetor $b$ de tamanho $4$, e resolva o sistema $A^{20}x = b$ por dois modos:
>     - Utilizando o método do passo 4;
>     - Calculando o produto $A^{20}=A\cdot A\cdots A$ com o algoritmo do passo 6, e resolvendo o sistema $(A^n)x = b$ diretamente com o algoritmo do passo 5.
>
>     Compare os números de operações e os resultados.

In [546]:
def decompoe_matriz_metodo_custoso (A, n, b):
    '''Função decompor matriz em produto LU e calcular o sistema A^n x = b
    
    Parametros obrigatorios
    ----------
    A : Array-like
        Matriz quadrada

    n : int
        Valor do expoente no cálculo

    b : Array-like de dimensão 1
        Vetor independente'''
    
    num_op_total = 0
    
    # Calcula A^n
    A_ = A.copy()
    for i in range(1, n):
        A_, num_op = matmul(A_, A)
        num_op_total += num_op

    # Separa A_ em LU
    L, U, num_op = crout_dolittle(A_)
    num_op_total += num_op

    
    y, num_op = solve_triangular_inferior(L, b)
    num_op_total += num_op

    x, num_op = solve_triangular_superior(U, y)
    num_op_total += num_op

    return x, num_op_total

In [547]:
ordem = 5
A = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
b = 20*np.random.rand(ordem) - 10 # Vetor aleatório com entradas em [-10,10]
n = 3

x, num_operacoes = decompoe_matriz_metodo_custoso(A, n, b)
print("Nº Op.: ", num_operacoes)
print("(A^3)x = b")
print(f"\n(A@A@A)x : \n", A@A@A@x)
print(f"\nb: \n",b)

Nº Op.:  630
(A^3)x = b

(A@A@A)x : 
 [-5.13181038 -6.17983444 -6.1686103   7.868822    2.05842426]

b: 
 [-5.13181038 -6.17983444 -6.1686103   7.868822    2.05842426]


> Vamos comparar os resultados pelos métodos da questão 4 e 7

In [548]:
num_op_4, num_op_7 = 0, 0
ordem = 4
n_max = 20
tabela = []

A = 20*np.random.rand(ordem,ordem) -10 # Matriz triangular aleatória com entradas em [-10,10]
b = 20*np.random.rand(ordem) - 10 # Vetor aleatório com entradas em [-10,10]

for n in range(1, n_max+1):
    _, num_op_4 = decompoe_matriz(A, n, b)
    _, num_op_7 = decompoe_matriz_metodo_custoso(A, n, b)

    tabela.append([n, num_op_4, num_op_7])


print(f"\nMatriz A: {ordem}x{ordem}")
print(f"\n|{'(M)':^10}|{'Nº Op.':^21}|")
print(f"|{'n':^10}|{'Metodo 4':^10}|{'Metodo 7':^10}|")

for linha in tabela:    
    print(f"|{linha[0]:^10}|{linha[1]:^10}|{linha[2]:^10}|")


Matriz A: 4x4

|   (M)    |       Nº Op.        |
|    n     | Metodo 4 | Metodo 7 |
|    1     |    72    |    72    |
|    2     |   104    |   200    |
|    3     |   136    |   328    |
|    4     |   168    |   456    |
|    5     |   200    |   584    |
|    6     |   232    |   712    |
|    7     |   264    |   840    |
|    8     |   296    |   968    |
|    9     |   328    |   1096   |
|    10    |   360    |   1224   |
|    11    |   392    |   1352   |
|    12    |   424    |   1480   |
|    13    |   456    |   1608   |
|    14    |   488    |   1736   |
|    15    |   520    |   1864   |
|    16    |   552    |   1992   |
|    17    |   584    |   2120   |
|    18    |   616    |   2248   |
|    19    |   648    |   2376   |
|    20    |   680    |   2504   |
